# Downloading pH glider data from the IOOS Glider Data Assembly Center

_Written by Lori Garzio, June 13, 2023_

[Rutgers Center for Ocean Observing Leadership](https://rucool.marine.rutgers.edu/) (RUCOOL)

This notebook demonstrates how to download NetCDF files from the [IOOS Glider DAC](https://gliders.ioos.us/) to a folder on your local machine using the [erddapy package](https://pypi.org/project/erddapy/). We will specify which variables to include in the file, and will be downloading the delayed-mode quality-controlled files from the [Glider DAC ERDDAP server](https://gliders.ioos.us/erddap/index.html). We have to download the delayed-mode datasets because real-time datasets do not have calculated pH included in the files. pH is calculated in the post-processing and QA/QC process, and those delayed-mode files are submitted to the DAC for public access.

In this example, we will be downloading delayed-mode data from the [spring 2021 deployment of ru30](https://gliders.ioos.us/erddap/tabledap/ru30-20210503T1929-delayed.html). You can browse this ERDDAP data page to get information about the deployment, make some graphs, and download the dataset in a variety of formats. However, instead of using this form to manually download the file, we are going to use python to download the dataset.

In [1]:
# import the required packages for data download
from erddapy import ERDDAP
import re
import os

The first step is to get a list of variables that are included in this dataset. The function below returns a list of variables for a user-specified ERDDAP server and dataset ID.

In [2]:
def get_dataset_variables(server, ds_id):
    e = ERDDAP(server=server,
               protocol='tabledap',
               response='nc')
    var_dict = e._get_variables(dataset_id=ds_id)

    return list(var_dict.keys())

In [3]:
ioos_server = 'https://gliders.ioos.us/erddap'
deployment_id = 'ru30-20210503T1929-delayed'

ds_vars = get_dataset_variables(ioos_server, deployment_id)
print(ds_vars)

['cdom', 'latitude_qc', 'pH_reference_voltage_corrected', 'time_qc', 'lat_uv_qc', 'precise_lon', 'salinity', 'salinity_interpolated', 'oxygen_saturation_corrected', 'salinity_lag_shifted', 'instrument_ctd', 'pressure', 'wmo_id', 'pH_raw', 'precise_time_qc', 'instrument_optode', 'chlorophyll_a', 'oxygen_saturation_raw', 'lon_uv_qc', 'time', 'u', 'precise_lat_qc', 'pressure_interpolated', 'lat_uv', 'oxygen_concentration_corrected', 'salinity_qc', 'instrument_flbbcd', 'v_qc', 'total_alkalinity', 'instrument_ph', 'precise_lon_qc', 'u_qc', 'conductivity_lag_shifted', 'longitude', 'temperature_qc', 'temperature_lag_shifted', 'profile_id', 'trajectory', 'depth_interpolated', 'pH_corrected', 'NC_GLOBAL', 'latitude', 'time_uv', 'conductivity_qc', 'aragonite_saturation_state', 'beta_700nm', 'conductivity', 'lon_uv', 'oxygen_concentration_raw', 'pH_reference_voltage_raw', 'platform_meta', 'density_qc', 'time_uv_qc', 'v', 'density', 'longitude_qc', 'temperature_interpolated', 'temperature', 'preci

Next, we'll filter down this list to generate a list of variables that we actually want to download, so we don't have a bunch of variables we don't need in our file.

In [4]:
search_vars = ['profile_id', 'latitude', 'longitude', 'depth', 'aragonite_saturation_state', 'chlorophyll', 
               'density', 'instrument_', 'oxygen', 'pH', 'pressure', 'salinity', 'temperature',
               'total_alkalinity']

In [5]:
r = re.compile('|'.join(search_vars))
new_list = list(filter(r.match, ds_vars))
print(new_list)

['latitude_qc', 'pH_reference_voltage_corrected', 'salinity', 'salinity_interpolated', 'oxygen_saturation_corrected', 'salinity_lag_shifted', 'instrument_ctd', 'pressure', 'pH_raw', 'instrument_optode', 'chlorophyll_a', 'oxygen_saturation_raw', 'pressure_interpolated', 'oxygen_concentration_corrected', 'salinity_qc', 'instrument_flbbcd', 'total_alkalinity', 'instrument_ph', 'longitude', 'temperature_qc', 'temperature_lag_shifted', 'profile_id', 'depth_interpolated', 'pH_corrected', 'latitude', 'aragonite_saturation_state', 'oxygen_concentration_raw', 'pH_reference_voltage_raw', 'density_qc', 'density', 'longitude_qc', 'temperature_interpolated', 'temperature', 'pressure_qc', 'density_lag_shifted', 'depth', 'depth_qc']


The new list looks better than the full list of dataset variables, but we don't need any of the QC variables since this dataset has already been QC'd (and we don't need to apply any more QC variables), so let's remove those QC variables. We also want to grab the 'time' variable.

In [6]:
var_list = [x for x in new_list if not x.endswith('_qc')]
print(var_list)

['pH_reference_voltage_corrected', 'salinity', 'salinity_interpolated', 'oxygen_saturation_corrected', 'salinity_lag_shifted', 'instrument_ctd', 'pressure', 'pH_raw', 'instrument_optode', 'chlorophyll_a', 'oxygen_saturation_raw', 'pressure_interpolated', 'oxygen_concentration_corrected', 'instrument_flbbcd', 'total_alkalinity', 'instrument_ph', 'longitude', 'temperature_lag_shifted', 'profile_id', 'depth_interpolated', 'pH_corrected', 'latitude', 'aragonite_saturation_state', 'oxygen_concentration_raw', 'pH_reference_voltage_raw', 'density', 'temperature_interpolated', 'temperature', 'density_lag_shifted', 'depth']


In [7]:
var_list.append('time')
print(var_list)

['pH_reference_voltage_corrected', 'salinity', 'salinity_interpolated', 'oxygen_saturation_corrected', 'salinity_lag_shifted', 'instrument_ctd', 'pressure', 'pH_raw', 'instrument_optode', 'chlorophyll_a', 'oxygen_saturation_raw', 'pressure_interpolated', 'oxygen_concentration_corrected', 'instrument_flbbcd', 'total_alkalinity', 'instrument_ph', 'longitude', 'temperature_lag_shifted', 'profile_id', 'depth_interpolated', 'pH_corrected', 'latitude', 'aragonite_saturation_state', 'oxygen_concentration_raw', 'pH_reference_voltage_raw', 'density', 'temperature_interpolated', 'temperature', 'density_lag_shifted', 'depth', 'time']


Now that we have the full list of variables that we want to include in our file, we can download our dataset. The function below will generate an xarray dataset for a user-specified ERDDAP server and dataset ID, with options to return selected variables and other constraints. In this example, we'll only be specifying the variables we want, not other constraints (which could be a specific time range, for example). You can find examples of specifying additional constraints in the [erddapy documentation](https://pypi.org/project/erddapy/).

In [8]:
def get_erddap_dataset(server, ds_id, variables=None, constraints=None):
    e = ERDDAP(server=server,
               protocol='tabledap',
               response='nc')
    
    e.dataset_id = ds_id
    
    if constraints:
        e.constraints = constraints
    if variables:
        e.variables = variables
        
    ds = e.to_xarray()
    ds = ds.sortby(ds.time)
    
    return ds

**Please Note:** The code in the next cell accesses the DAC ERDDAP server and tries to return all of the data you requested. Because we're requesting a high-resolution delayed-mode dataset, sometimes you will get a really long error message when you try to run this next block of code. If the top of the error reads: "TimeoutError" and the bottom reads: "ReadTimeout: The read operation timed out", try to run the code again. You might need to run it several times to finally get your dataset because although the operation may time out, your request will be cached and it will eventually be fulfilled.

In [10]:
kwargs = dict()
kwargs['variables'] = var_list
ds = get_erddap_dataset(ioos_server, deployment_id, **kwargs)

In [11]:
ds

<xarray.Dataset>
Dimensions:                         (trajectory: 1, profile: 7405, obs: 2171361)
Coordinates:
  * trajectory                      (trajectory) object 'ru30-20210503T1929'
    longitude                       (profile) float64 ...
    latitude                        (profile) float64 ...
    time                            (profile) datetime64[ns] ...
    depth                           (obs) float32 ...
Dimensions without coordinates: profile, obs
Data variables: (12/29)
    profile_id                      (profile) float64 ...
    trajectoryIndex                 (profile) int32 ...
    rowSize                         (profile) int32 ...
    pH_reference_voltage_corrected  (obs) float32 ...
    salinity                        (obs) float32 ...
    salinity_interpolated           (obs) float32 ...
    ...                              ...
    oxygen_concentration_raw        (obs) float32 ...
    pH_reference_voltage_raw        (obs) float32 ...
    density                         (obs) float32 ...
    temperature_interpolated        (obs) float32 ...
    temperature                     (obs) float32 ...
    density_lag_shifted             (obs) float32 ...
Attributes: (12/59)
    acknowledgment:                This deployment supported by the NOAA Ocea...
    cdm_data_type:                 TrajectoryProfile
    cdm_profile_variables:         time_uv,lat_uv,lon_uv,u,v,profile_id,time,...
    cdm_trajectory_variables:      trajectory,wmo_id
    comment:                       Glider operated by the Rutgers University ...
    contributor_name:              Grace Saba, Elizabeth Wright-Fairbanks, Th...
    ...                            ...
    subsetVariables:               wmo_id,trajectory,profile_id,time,latitude...
    summary:                       This project is conducting seasonal deploy...
    time_coverage_end:             2021-05-27T14:30:22.34Z
    time_coverage_start:           2021-05-03T19:32:52.50Z
    title:                         ru30-20210503T1929-delayed
    Westernmost_Easting:           -74.57585

Now specify a location on your local machine to save the NetCDF file. The lines of code below will generate a new folder with the deployment_id in your specified save location, and will save the netcdf in that directory.

In [12]:
save_dir = '/Users/garzio/Documents/rucool/gliderdata'
save_dir = os.path.join(save_dir, deployment_id)
os.makedirs(save_dir, exist_ok=True)
fname = f'{deployment_id}.nc'
ds.to_netcdf(os.path.join(save_dir, fname))
print(f'File saved to: {os.path.join(save_dir, fname)}')

File saved to: /Users/garzio/Documents/rucool/gliderdata/ru30-20210503T1929-delayed/ru30-20210503T1929-delayed.nc
